In [1]:
import sys
from pathlib import Path
from multiprocessing import Pool
import pandas as pd

sys.path.append('../src')
from utils import run_cmd
from parse import parse_sistr_result

In [2]:
def run_sistr(filepath, sample_name, output, threads=1):
    run_cmd(f'conda run -n sistr sistr -i {filepath} {sample_name} -f json -o {output} -T /dev/shm -t {threads}')

In [3]:
with open('/media/Central_Lab_Storage/NcbiASM/Salmonella_enterica/selected.txt') as f:
    accs = set(f.read().splitlines())

In [4]:
dirpath = Path('/media/Central_Lab_Storage/NcbiASM/Salmonella_enterica/Contigs')
outpath = Path('/media/Central_Lab_Storage/NcbiASM/Salmonella_enterica/SISTR')

In [5]:
with Pool(72) as p:
    try:
        for i in dirpath.iterdir():
            if i.stem in accs:
                output = outpath/i.stem
                p.apply_async(run_sistr, (i, i.stem, output))
        p.close()
        p.join()
    except KeyboardInterrupt:
        p.terminate()

In [6]:
dirpath = Path('/media/Central_Lab_Storage/NcbiASM/Salmonella_enterica/SISTR')
summaries = {i.stem: parse_sistr_result(i) for i in dirpath.iterdir()}

In [8]:
df = pd.DataFrame(summaries).T
df = df.sort_index()
df.index.name = 'Key'
df.head()

,subspecies,serotype,serogroup
Key,,,
GCA_000006945.2,enterica,Typhimurium,B
GCA_000007545.1,enterica,Typhi,D1
GCA_000008105.1,enterica,Choleraesuis,C1
GCA_000009505.1,enterica,Enteritidis,D1
GCA_000009525.1,enterica,Gallinarum,D1


In [9]:
df.to_csv('/media/Central_Lab_Storage/NcbiASM/Salmonella_enterica/SISTR.tsv', sep='\t')